# An overview of QuantTensor and QuantConv2d

In this initial tutorial, we take a first look at `QuantTensor`, a basic data structure in Brevitas, and at `QuantConv2d`, a typical quantized layer. `QuantConv2d` is an instance of a `QuantWeightBiasInputOutputLayer` (typically imported as `QuantWBIOL`), meaning that it supports quantization of its weight, bias, input and output. Other instances of `QuantWBIOL` are `QuantLinear`, `QuantConv1d`, `QuantConvTranspose1d` and `QuantConvTranspose2d`, and they all follow the same principles.

If we take a look at the `__init__` method of `QuantConv2d`, we notice a few things:

In [1]:
import inspect
from brevitas.nn import QuantConv2d
from IPython.display import Markdown, display

def pretty_print_source(source):
    display(Markdown('```python\n' + source + '\n```'))
    
source = inspect.getsource(QuantConv2d.__init__)  
pretty_print_source(source)

No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v10.2'


```python
    def __init__(
            self,
            in_channels: int,
            out_channels: int,
            kernel_size: Union[int, Tuple[int, int]],
            stride: Union[int, Tuple[int, int]] = 1,
            padding: Union[int, Tuple[int, int]] = 0,
            dilation: Union[int, Tuple[int, int]] = 1,
            groups: int = 1,
            bias: bool = True,
            padding_type: str = 'standard',
            weight_quant: Union[WeightQuantProxyProtocol, Type[Injector]] = Int8WeightPerTensorFloat,
            bias_quant: Union[BiasQuantProxyProtocol, Type[Injector]] = None,
            input_quant: Union[ActQuantProxyProtocol, Type[Injector]] = None,
            output_quant: Union[ActQuantProxyProtocol, Type[Injector]] = None,
            return_quant_tensor: bool = False,
            **kwargs) -> None:
        Conv2d.__init__(
            self,
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
            groups=groups,
            bias=bias)
        QuantWBIOL.__init__(
            self,
            weight=self.weight,
            bias=self.bias,
            weight_quant=weight_quant,
            bias_quant=bias_quant,
            input_quant=input_quant,
            output_quant=output_quant,
            return_quant_tensor=return_quant_tensor,
            **kwargs)
        assert self.padding_mode == 'zeros'
        assert not (padding_type == 'same' and padding != 0)
        self.padding_type = padding_type

```

`QuantConv2d` is an instance of both `Conv2d` and `QuantWBIOL`. Its initialization method exposes the usual arguments of a `Conv2d`, as well as: an extra flag to support *same padding*; *four* different arguments to set a quantizer for - respectively - *weight*, *bias*, *input*, and *output*; a `return_quant_tensor` boolean flag; the `**kwargs` placeholder to intercept additional arbitrary keyword arguments.  
In this tutorial we will focus on how to set the four quantizer arguments and the return flags; arbitrary kwargs will be explained in a separate tutorial dedicated to defining and overriding quantizers.

By default `weight_quant=Int8WeightPerTensorFloat`, while `bias_quant`, `input_quant` and `output_quant` are set to `None`. That means that by default weights are quantized to *8-bit signed integer with a per-tensor floating-point scale factor* (a very common type of quantization adopted by e.g. the ONNX standard opset), while quantization of bias, input, and output are disabled. We can easily verify all of this at runtime on an example:

In [2]:
default_quant_conv = QuantConv2d(
    in_channels=2, out_channels=3, kernel_size=(3,3), bias=False)

In [3]:
print(f'Is weight quant enabled: {default_quant_conv.is_weight_quant_enabled}')
print(f'Is bias quant enabled: {default_quant_conv.is_bias_quant_enabled}')
print(f'Is input quant enabled: {default_quant_conv.is_input_quant_enabled}')
print(f'Is output quant enabled: {default_quant_conv.is_output_quant_enabled}')

Is weight quant enabled: True
Is bias quant enabled: False
Is input quant enabled: False
Is output quant enabled: False


If we now try to pass in a random floating-point tensor as input, as expected we get the output of the convolution:

In [4]:
import torch

out = default_quant_conv(torch.randn(1, 2, 5, 5))
out

tensor([[[[-0.2946, -0.9876, -0.6115],
          [ 0.1994,  0.5248, -0.7628],
          [-0.2923, -0.3050, -0.2904]],

         [[ 0.3183, -2.0555,  0.6976],
          [ 0.3898,  0.2366,  0.3024],
          [ 0.6329, -0.5668, -0.9263]],

         [[ 0.0127, -0.7625,  0.6033],
          [ 0.0046, -0.4024,  0.3471],
          [ 1.0292, -0.3654,  0.2626]]]], grad_fn=<ThnnConv2DBackward>)

In this case we are computing the convolution between an unquantized input tensor and quantized weights, so the output in general is unquantized.

A QuantConv2d with quantization disabled everywhere behaves like a standard `Conv2d`. Again can easily verify this:

In [5]:
from torch.nn import Conv2d

torch.manual_seed(0)  # set a seed to make sure the random weight init is reproducible
disabled_quant_conv = QuantConv2d(
    in_channels=2, out_channels=3, kernel_size=(3,3), bias=False, weight_quant=None)
torch.manual_seed(0)  # reproduce the same random weight init as above
float_conv = Conv2d(
    in_channels=2, out_channels=3, kernel_size=(3,3), bias=False)
inp = torch.randn(1, 2, 5, 5)
torch.isclose(disabled_quant_conv(inp), float_conv(inp)).all().item()

True

As we have just seen, Brevitas allows users as much freedom as possible to experiment with quantization, meaning that computation between quantized and unquantized values is considered legal. This allows users to mix Brevitas layers with Pytorch layers with little restrictions.  
To make this possible, quantized values are typically represented in *dequantized format*, meaning that - in the case of affine quantization implemented in Brevitas - zero-point and scale factor are applied to their integer values according to the formula **quant_value = (integer_value - zero_point) * scale**.

## QuantTensor

We can directly observe the quantized weights by calling the weight quantizer on the layer's weights: `default_quant_conv.weight_quant(quant_conv.weight)`, which for shortness is already implemented as `default_quant_conv.quant_weight()` :

In [6]:
default_quant_conv.quant_weight()

QuantTensor(value=tensor([[[[-0.2269, -0.2177,  0.1919],
          [ 0.0037,  0.2269, -0.0793],
          [ 0.1144,  0.0849, -0.2011]],

         [[ 0.0314,  0.2251, -0.0719],
          [ 0.0406,  0.1181,  0.2011],
          [-0.0978,  0.0517,  0.0055]]],


        [[[-0.2306,  0.0332,  0.0849],
          [ 0.1845,  0.0406, -0.2287],
          [-0.1494,  0.1070, -0.0387]],

         [[-0.1476, -0.1347,  0.1771],
          [ 0.1845, -0.2029,  0.0221],
          [-0.0332,  0.1531,  0.0018]]],


        [[[-0.2343, -0.1476, -0.2103],
          [-0.0996,  0.0904,  0.0627],
          [-0.2140,  0.0258,  0.1955]],

         [[-0.1439, -0.0277,  0.1236],
          [ 0.1365, -0.1439, -0.2195],
          [ 0.0719,  0.1771,  0.0221]]]], grad_fn=<MulBackward0>), scale=tensor(0.0018, grad_fn=<DivBackward0>), zero_point=tensor(0.), bit_width=tensor(8.), signed=True, training=True)

Notice how the quantized weights are wrapped in a data structure implemented by Brevitas called `QuantTensor`. A `QuantTensor` is a way to represent an affine quantized tensor with all its metadata, meaning: the `value` of the quantized tensor in *dequantized* format, `scale`, `zero_point`, `bit_width`, whether the quantized value it's `signed` or not, and whether the tensor was generated in `training` mode. 

**Note**: currently `QuantTensor` is not a subclass of `torch.Tensor`, as this is something that Pytorch started to support only recently, so in general calling a `torch` operator on a `QuantTensor` will return an exception. Support for this kind of tighter integration between Brevitas and Pytorch is planned. 

As expected, we have that the quantized value (in dequantized format) can be computer from its integer representation, together with zero-point and scale:

In [7]:
int_weight = default_quant_conv.int_weight()
zero_point = default_quant_conv.quant_weight_zero_point()
scale = default_quant_conv.quant_weight_scale()
quant_weight_manually = (int_weight - zero_point) * scale
default_quant_conv.quant_weight().value.isclose(quant_weight_manually).all().item()

True

A *valid* QuantTensor correctly populates all its fields with values `!= None` and respect the **affine quantization invariant**, i.e. `value / scale + zero_point` is (accounting for rounding errors) an *integer* that can be represented within the interval defined by the `bit_width` and `signed` fields of the `QuantTensor`. A *non-valid* one doesn't.
We can observe that the quantized weights are indeed marked as valid:

In [8]:
default_quant_conv.quant_weight().is_valid

True

Calling `is_valid` is relative expensive, so it should be using sparingly, but there are a few cases where a non-valid QuantTensor might be generated that is important to be aware of. Say we instantiate the layer again, this time with `return_quant_tensor=True`:

In [9]:
return_quant_conv = QuantConv2d(
    in_channels=2, out_channels=3, kernel_size=(3,3), bias=False, return_quant_tensor=True)

We then again pass as input a random floating-point tensor. Because `input_quant=None` and `output_quant=None` (i.e. both input and output quantization are disabled), again as before we are performing a convolution between a quantized and an unquantized tensor, which in general returns an unquantized tensor:

In [10]:
out_tensor = return_quant_conv(torch.randn(1, 2, 5, 5))
out_tensor

QuantTensor(value=tensor([[[[-0.6238, -0.1567,  0.5639],
          [-0.3426,  0.0662,  0.6296],
          [-0.6507,  0.4468, -0.4465]],

         [[ 1.0313,  0.7856, -0.2931],
          [-0.6213,  0.5228,  0.7288],
          [ 0.1397,  0.0216,  0.7518]],

         [[ 0.0763, -0.3561, -0.0491],
          [-0.5127,  0.2945,  0.5501],
          [-0.2460, -0.0052,  0.0395]]]], grad_fn=<ThnnConv2DBackward>), scale=None, zero_point=None, bit_width=None, signed=None, training=True)

Because we set `return_quant_tensor=True`, we get a `QuantTensor` as output object. However, we observe that `scale`, `zero_point` and `bit_width` of the output `QuantTensor` are set to `None`. This is expected since the output tensor is unquantized. In this case then the `QuantTensor` is really just acting as a wrapper around a `torch.Tensor`, and as such is market as non-valid.

In [11]:
out_tensor.is_valid

False

## Input Quantization

We can obtain a valid output `QuantTensor` by making sure that both input and weight of `QuantConv2d` are quantized. To do so, we can set a quantizer for `input_quant`. In this example we pick a *signed 8-bit* quantizer with *per-tensor floating-point scale factor*:

In [12]:
from brevitas.quant.scaled_int import Int8ActPerTensorFloat

input_quant_conv = QuantConv2d(
    in_channels=2, out_channels=3, kernel_size=(3,3), bias=False, 
    input_quant=Int8ActPerTensorFloat, return_quant_tensor=True)
out_tensor = input_quant_conv(torch.randn(1, 2, 5, 5))
out_tensor

QuantTensor(value=tensor([[[[ 0.1924, -0.3169,  0.5825],
          [ 0.8168, -0.5471,  0.6359],
          [ 1.0381,  0.2220, -0.0910]],

         [[-0.0771,  0.4335,  0.4567],
          [ 0.5606,  0.8164,  0.3263],
          [ 0.0904,  0.0572,  0.2695]],

         [[ 0.2439, -0.3544, -0.5951],
          [-0.0675, -0.1017, -0.3615],
          [-0.8483,  0.9859,  0.1841]]]], grad_fn=<ThnnConv2DBackward>), scale=tensor([[[[3.3017e-05]]]], grad_fn=<MulBackward0>), zero_point=tensor(0.), bit_width=tensor(21.), signed=True, training=True)

In [13]:
out_tensor.is_valid

True

What happens internally is that the input tensor passed to `input_quant_conv` is being quantized before being passed to the convolution operator. That means we are now computing a convolution between two quantized tensors, which mimplies that the output of the operation is also quantized. As expected then `out_tensor` is marked as valid. 

Another important thing to notice is how the `bit_width` field of `out_tensor` is relatively high at *21 bits*. In Brevitas, the assumption is always that the output bit-width of an operator reflects the worst-case size of the *accumulator* required by that operation. In other terms, given the *size* of the input and weight tensors and their *bit-widths*, 21 is the bit-width that would be required to represent the largest possible output value that could be generated. This makes sure that the affine quantization invariant is always respected.

We could have obtained a similar result by directly passing as input a QuantTensor. In this example we are directly defining a QuantTensor ourselves, but it could also be the output of a previous layer.

In [14]:
from brevitas.quant_tensor import QuantTensor

scale = 0.0001
bit_width = 8
zero_point = 0.
int_value = torch.randint(low=- 2 ** (bit_width - 1), high=2 ** (bit_width - 1) - 1, size=(1, 2, 5, 5))
quant_value = (int_value - zero_point) * scale
quant_tensor_input = QuantTensor(
    quant_value, 
    scale=torch.tensor(scale), 
    zero_point=torch.tensor(zero_point), 
    bit_width=torch.tensor(float(bit_width)),
    signed=True,
    training=True)
quant_tensor_input

QuantTensor(value=tensor([[[[-0.0087, -0.0034, -0.0026, -0.0038, -0.0062],
          [-0.0073,  0.0082, -0.0019,  0.0087, -0.0060],
          [-0.0111, -0.0027, -0.0121, -0.0098, -0.0111],
          [-0.0023,  0.0092,  0.0078, -0.0034, -0.0041],
          [-0.0122, -0.0093, -0.0067, -0.0073, -0.0009]],

         [[ 0.0018,  0.0036, -0.0096,  0.0057,  0.0025],
          [-0.0124, -0.0072,  0.0037, -0.0023,  0.0007],
          [-0.0127,  0.0052,  0.0004, -0.0079,  0.0095],
          [ 0.0066,  0.0054,  0.0025,  0.0111,  0.0024],
          [ 0.0100, -0.0037,  0.0072, -0.0115, -0.0058]]]]), scale=tensor(1.0000e-04), zero_point=tensor(0.), bit_width=tensor(8.), signed=True, training=True)

In [15]:
quant_tensor_input.is_valid

True

**Note**: how we are explicitly forcing `value`, `scale`, `zero_point` and `bit_width` to be floating-point `torch.Tensor`, as this is expected by Brevitas but it's currently not enforced automatically at initialization time.

If we now pass in `quant_tensor_input` to `return_quant_conv`, we will see that indeed the output is a valid 21-bit `QuantTensor`:

In [16]:
out_tensor = return_quant_conv(quant_tensor_input)
out_tensor

QuantTensor(value=tensor([[[[ 0.0010,  0.0033,  0.0028],
          [ 0.0039,  0.0010,  0.0044],
          [ 0.0004,  0.0023,  0.0040]],

         [[ 0.0042,  0.0031, -0.0031],
          [ 0.0020, -0.0066, -0.0035],
          [-0.0037,  0.0028, -0.0049]],

         [[ 0.0079,  0.0039,  0.0037],
          [ 0.0021, -0.0035,  0.0044],
          [ 0.0010,  0.0062,  0.0010]]]], grad_fn=<ThnnConv2DBackward>), scale=tensor([[[[1.7899e-07]]]], grad_fn=<MulBackward0>), zero_point=tensor(0.), bit_width=tensor(21.), signed=True, training=True)

In [17]:
out_tensor.is_valid

True

We can also pass in an input `QuantTensor` to a layer that has `input_quant` enabled. In that case, the input gets re-quantized:

In [18]:
input_quant_conv(quant_tensor_input)

QuantTensor(value=tensor([[[[-0.0044, -0.0023,  0.0052],
          [-0.0016,  0.0025,  0.0021],
          [-0.0001, -0.0040,  0.0011]],

         [[ 0.0018,  0.0067,  0.0013],
          [-0.0078, -0.0063, -0.0073],
          [ 0.0035,  0.0033,  0.0046]],

         [[-0.0012, -0.0002, -0.0019],
          [ 0.0008,  0.0021, -0.0023],
          [-0.0057,  0.0019,  0.0009]]]], grad_fn=<ThnnConv2DBackward>), scale=tensor([[[[1.8251e-07]]]], grad_fn=<MulBackward0>), zero_point=tensor(0.), bit_width=tensor(21.), signed=True, training=True)

## Output Quantization

Let's now look at would have happened if we instead enabled output quantization:

In [19]:
from brevitas.quant.scaled_int import Int8ActPerTensorFloat

output_quant_conv = QuantConv2d(
    in_channels=2, out_channels=3, kernel_size=(3,3), bias=False, 
    output_quant=Int8ActPerTensorFloat, return_quant_tensor=True)
out_tensor = output_quant_conv(torch.randn(1, 2, 5, 5))
out_tensor

QuantTensor(value=tensor([[[[ 0.0424,  0.3899, -0.0254],
          [ 0.4832, -0.0509,  0.6103],
          [-0.0763, -0.4408,  0.4493]],

         [[ 0.3984,  0.1611, -1.0850],
          [-0.0848,  0.6103, -0.9155],
          [ 0.3730, -0.7883,  0.0509]],

         [[-0.3814, -0.5595, -0.0339],
          [-0.3984, -0.8477,  0.0848],
          [ 0.9918,  0.8731, -0.3730]]]], grad_fn=<MulBackward0>), scale=tensor(0.0085, grad_fn=<DivBackward0>), zero_point=tensor(0.), bit_width=tensor(8.), signed=True, training=True)

In [20]:
out_tensor.is_valid

True

We can see again that the output is a valid `QuantTensor`. However, what happened internally is quite different from before.  
Previously, we computed the convolution between two quantized tensors, and got a quantized tensor as output.  
In this case instead, we compute the convolution between a quantized and an unquantized tensor, we take its unquantized output and we quantize it.  
The difference is obvious once we look at the output `bit_width`. In the previous case, we had that the `bit_width` reflected the size of the output accumulator. In this case instead, we have `bit_width=tensor(8.)`, which is what we expected since `output_quant` had been set to an *Int8* quantizer.

## Bias Quantization

There is an important scenario where the various options we just saw make a practical difference, and it's quantization of *bias*. In many contexts, such as in the ONNX standard opset and in FINN, bias is assumed to be quantized with scale factor equal to *input scale * weight scale*, which means that we need a valid quantized input somehow. A predefined bias quantizer that reflects that assumption is `brevitas.quant.scaled_int.Int8Bias`. If we simply tried to set it to a `QuantConv2d` without any sort of input quantization, we would get an error:

In [21]:
from brevitas.quant.scaled_int import Int8Bias

bias_quant_conv = QuantConv2d(
    in_channels=2, out_channels=3, kernel_size=(3,3), bias=True,
    bias_quant=Int8Bias, return_quant_tensor=True)
bias_quant_conv(torch.randn(1, 2, 5, 5))

RuntimeError: Input scale required

We can solve the issue by passing in a valid `QuantTensor`, e.g. the `quant_tensor_input`  we defined above:

In [22]:
bias_quant_conv(quant_tensor_input)

QuantTensor(value=tensor([[[[-0.0029,  0.0032,  0.0021],
          [ 0.0022,  0.0005,  0.0075],
          [ 0.0009, -0.0051,  0.0009]],

         [[ 0.0062, -0.0039,  0.0084],
          [ 0.0082, -0.0004,  0.0051],
          [ 0.0068, -0.0069, -0.0029]],

         [[ 0.0025,  0.0026,  0.0015],
          [-0.0102, -0.0056, -0.0106],
          [ 0.0016,  0.0045,  0.0023]]]], grad_fn=<ThnnConv2DBackward>), scale=tensor([[[[1.8527e-07]]]], grad_fn=<MulBackward0>), zero_point=tensor(0.), bit_width=tensor(22.), signed=True, training=True)

Or by enabling input quantization and then passing in a float a `torch.Tensor` or a `QuantTensor`:

In [23]:
input_bias_quant_conv = QuantConv2d(
    in_channels=2, out_channels=3, kernel_size=(3,3), bias=True,
    input_quant=Int8ActPerTensorFloat, bias_quant=Int8Bias, return_quant_tensor=True)
input_bias_quant_conv(torch.randn(1, 2, 5, 5))

QuantTensor(value=tensor([[[[-0.3430, -0.3023,  0.7517],
          [ 0.0345, -0.0177,  1.8919],
          [-0.1800,  0.2647, -1.0650]],

         [[ 0.4444, -0.2866, -0.2996],
          [ 0.4732,  0.3038,  0.1380],
          [ 0.6215,  0.4520,  0.3587]],

         [[ 0.3597, -0.4442, -0.4382],
          [ 0.4902, -0.0222, -0.1807],
          [-0.3392,  0.5933, -0.9689]]]], grad_fn=<ThnnConv2DBackward>), scale=tensor([[[[3.6648e-05]]]], grad_fn=<MulBackward0>), zero_point=tensor(0.), bit_width=tensor(22.), signed=True, training=True)

In [24]:
input_bias_quant_conv(quant_tensor_input)

QuantTensor(value=tensor([[[[ 3.3137e-03,  3.2802e-03, -8.9473e-04],
          [-4.6457e-03, -7.7606e-03,  3.5226e-03],
          [ 5.6110e-05,  1.4331e-02,  6.0171e-03]],

         [[ 1.0900e-03, -2.6452e-04,  1.4820e-04],
          [-2.1026e-03, -2.9498e-03, -3.8433e-03],
          [-1.6176e-03,  2.2537e-03,  3.0565e-03]],

         [[ 8.0153e-03, -1.8817e-03,  5.5593e-03],
          [-5.7597e-03, -4.9193e-03, -2.5435e-03],
          [ 2.7699e-03,  7.0996e-03, -3.6194e-03]]]],
       grad_fn=<ThnnConv2DBackward>), scale=tensor([[[[1.7813e-07]]]], grad_fn=<MulBackward0>), zero_point=tensor(0.), bit_width=tensor(22.), signed=True, training=True)

Notice how the output `bit_width=tensor(22.)`. This is because, in the worst-case, summing a *21-bit* integer (the size of the accumulator before bias is added) and an *8-bit* integer (the size of quantized bias) gives a *22-bit* integer.

Let's try now to enable output quantization instead of input quantization. That wouldn't have solved the problem with bias quantization, as output quantization is performed after bias is added:

In [25]:
output_bias_quant_conv = QuantConv2d(
    in_channels=2, out_channels=3, kernel_size=(3,3), bias=True,
    output_quant=Int8ActPerTensorFloat, bias_quant=Int8Bias, return_quant_tensor=True)
output_bias_quant_conv(torch.randn(1, 2, 5, 5))

RuntimeError: Input scale required

Not all scenarios require bias quantization to depend on the scale factor of the input. In those cases, biases can be quantized the same way weights are quantized, and have their own scale factor. In Brevitas, a predefined quantizer that reflects this other scenario is `Int8BiasPerTensorFloatInternalScaling`. In this case then a valid quantized input is not required:

In [26]:
from brevitas.quant.scaled_int import Int8BiasPerTensorFloatInternalScaling

bias_internal_scale_quant_conv = QuantConv2d(
    in_channels=2, out_channels=3, kernel_size=(3,3), bias=True,
    bias_quant=Int8BiasPerTensorFloatInternalScaling, return_quant_tensor=True)
bias_internal_scale_quant_conv(torch.randn(1, 2, 5, 5))

QuantTensor(value=tensor([[[[ 0.2789, -0.3630,  0.3842],
          [-0.3696, -0.3159, -0.7956],
          [-0.1516,  0.0332, -1.1296]],

         [[ 0.9501, -0.0195,  0.8198],
          [-1.0088,  0.5263,  0.9647],
          [ 0.1448, -0.5011, -0.7708]],

         [[ 0.3887, -0.3169,  0.2874],
          [ 0.0871,  0.2122,  0.6736],
          [ 1.0793, -0.4930,  1.4344]]]], grad_fn=<ThnnConv2DBackward>), scale=None, zero_point=None, bit_width=None, signed=None, training=True)

There are a couple of situations to be aware of concerning bias quantization that can lead to changes in the output `zero_point`.

Let's consider the scenario where we compute the convolution between a quantized input tensor and quantized weights. In the first case, we then add an *unquantized* bias on top of the output. In the second one, we add a bias quantized with its own scale factor, e.g. with the `Int8BiasPerTensorFloatInternalScaling` quantizer. In both cases, in order to make sure the output `QuantTensor` is valid (i.e. the affine quantization invariant is respected), the output `zero_point` becomes non-zero:

In [27]:
unquant_bias_input_quant_conv = QuantConv2d(
    in_channels=2, out_channels=3, kernel_size=(3,3), bias=True,
    input_quant=Int8ActPerTensorFloat, return_quant_tensor=True)
out_tensor = unquant_bias_input_quant_conv(torch.randn(1, 2, 5, 5))
out_tensor

QuantTensor(value=tensor([[[[-0.7197,  0.0986,  0.0688],
          [-0.0811, -0.2854, -0.0884],
          [-0.2002, -0.6859, -0.4805]],

         [[ 1.0232, -0.0911,  0.3650],
          [ 0.9476,  0.1415, -0.0981],
          [ 0.1540,  0.0060,  0.2123]],

         [[ 0.1468, -0.1109, -0.1193],
          [-0.1423,  0.2657, -0.7881],
          [-0.1250,  0.5390,  1.1821]]]], grad_fn=<ThnnConv2DBackward>), scale=tensor([[[[3.5363e-05]]]], grad_fn=<MulBackward0>), zero_point=tensor([[[[ 6384.8540]],

         [[ 1574.9872]],

         [[-5444.3950]]]], grad_fn=<DivBackward0>), bit_width=tensor(21.), signed=True, training=True)

In [28]:
out_tensor.is_valid

True

Finally, an important point about `QuantTensor`. With the exception of learned bit-width (which will be the subject of a separate tutorial) and some of the bias quantization scenarios we have just seen, usually returing a `QuantTensor` is not necessary and - because `QuantTensor` is not yet a subclass of `torch.Tensor` - it can actually easily lead to errors when the next layer is not a quantized Brevitas layer.  
Returning the output quantized tensor in dequantized format in a `torch.Tensor` datatype is typically enough. This is why currently `return_quant_tensor` defaults to `False`. We can easily see it in an example:

In [29]:
bias_input_quant_conv = QuantConv2d(
    in_channels=2, out_channels=3, kernel_size=(3,3), bias=True,
    input_quant=Int8ActPerTensorFloat, bias_quant=Int8Bias)
bias_input_quant_conv(torch.randn(1, 2, 5, 5))

tensor([[[[ 0.5123, -0.4353, -0.3736],
          [-0.6345, -0.4745, -0.2858],
          [-1.3498,  0.9756,  0.9841]],

         [[ 0.4601, -0.4752, -0.2999],
          [ 0.0640, -0.6955,  0.2470],
          [-0.3436, -0.4508,  0.6135]],

         [[-0.2197,  0.1174, -0.4480],
          [-0.7926,  0.0382,  0.8281],
          [ 0.9191,  0.2873, -0.1213]]]], grad_fn=<ThnnConv2DBackward>)

Altough not obvious, the output is actually implicitly quantized.